In [1]:
import subprocess
from distutils.version import LooseVersion

import dxdata
import dxpy
import pyspark
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

In [2]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

dispensed_database_name = dxpy.find_one_data_object(
    classname="database", name="app*", folder="/", name_mode="glob", describe=True
)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)["id"]

dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]

In [7]:
def fields_for_id(field_id):

    field_id = str(field_id)
    fields = participant.find_fields(
        name_regex=r"^p{}(_i\d+)?(_a\d+)?$".format(field_id)
    )

    return sorted(fields, key=lambda f: LooseVersion(f.name))

fields = [
    "21022",
    "22001",
    "22009",
    "26216",
]

field_names = [fields_for_id(id) for id in fields]
field_names = ["eid"] + [field.name for fields in field_names for field in fields]

pcs = {f"p22009_a{i}": f"PC{i}" for i in range(1, 21)}
covs = ["FID", "IID", "SEX", "AGE", "AGE2", "AGESEX", "AGE2SEX", "BMI_STD_PRS"] + list(pcs.values())

In [8]:
covs

['FID',
 'IID',
 'SEX',
 'AGE',
 'AGE2',
 'AGESEX',
 'AGE2SEX',
 'BMI_STD_PRS',
 'PC1',
 'PC2',
 'PC3',
 'PC4',
 'PC5',
 'PC6',
 'PC7',
 'PC8',
 'PC9',
 'PC10',
 'PC11',
 'PC12',
 'PC13',
 'PC14',
 'PC15',
 'PC16',
 'PC17',
 'PC18',
 'PC19',
 'PC20']

In [10]:
df = participant.retrieve_fields(
    names=field_names, engine=dxdata.connect(), coding_values="raw"
)

df = df.na.drop(how="any")

df = (
    df.select([F.col(c).alias(pcs.get(c, c)) for c in df.columns])
    .withColumn("FID", F.col("eid"))
    .withColumn("IID", F.col("eid"))
    .withColumn("SEX", F.col("p22001").cast(IntegerType()))
    .withColumn("AGE", F.col("p21022").cast(IntegerType()))
    .withColumn("AGE2", (F.col("p21022") ** 2).cast(IntegerType()))
    .withColumn("AGESEX", (F.col("p21022") * F.col("p22001")).cast(IntegerType()))
    .withColumn(
        "AGE2SEX", ((F.col("p21022") ** 2) * F.col("p22001")).cast(IntegerType())
    )
    .withColumn("BMI_STD_PRS", F.col("p26216"))
    .select(*covs)
)

In [11]:
df.coalesce(1).write.csv(
    "/tmp/covariates.tsv",
    sep="\t",
    header=True,
)

In [12]:
subprocess.run(
    ["hadoop", "fs", "-getmerge", "/tmp/covariates.tsv", "../tmp/covariates.tsv"],
    check=True,
    shell=False,
)
subprocess.run(
    ["dx", "upload", "../tmp/covariates.tsv", "--path", "/Data/phenotypes/"],
    check=True,
    shell=False,
)

CompletedProcess(args=['dx', 'upload', '../tmp/covariates.tsv', '--path', '/Data/phenotypes/'], returncode=0)